In [1]:
base_path = "/content/drive/My Drive/yl_tez"

In [2]:
import os
if 'COLAB_GPU' in os.environ:
    from google.colab import drive
    drive.mount('/content/drive')
else:
    print("Yerelde Çalışıyor.")
    base_path = "."

Mounted at /content/drive


In [3]:
!pip install datasets
!pip install trl

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 547.8/547.8 kB 7.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.8/40.8 MB 39.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 21.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.9/64.9 kB 10.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 34.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 25.1 MB/s eta 0:00:00
  Attempting uninstall: requests
    Found existing installation: requests 2.31.0
    Uninstalling requests-2.31.0:
      Successfully uninstalled requests-2.31.0
  Attempting uninstall: pyarrow
    Found existing installation: pyarrow 14.0.2
    Uninstalling pyarrow-14.0.2:
      Successfully uninstalled pyarrow-14.0.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cudf-cu12 

In [4]:
import pandas as pd
from transformers import AutoModelForCausalLM, AutoTokenizer
from datasets import Dataset
from transformers import TrainingArguments
from trl import SFTTrainer, SFTConfig

In [5]:
model_adi = "BestModelParameters"
model_yolu = os.path.join(base_path, model_adi)
veri_seti_adi =  "ana_veri_seti_deneme.csv"
data_filepath = os.path.join(base_path, veri_seti_adi)
SORU_TOKEN ="<|soru|>"
CEVAP_TOKEN ="<|cevap|>"

In [6]:
# Veri dosyasını oku
df = pd.read_csv(data_filepath)

In [7]:
print(df.head().to_markdown())

|    | text                                                                                                                                                                                                                                                                                                                                         |
|---:|:---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------|
|  0 | Durum: Ben hipertansiyon hastası değilim.  Bende kalp hastalığı yok.  Benim kilom biraz fazla.  Ben yaşlı bir insanım.  Ben bir kadınım.  Ben hayatım boyunca asla sigara içmedim.  Ben HbA1c seviyeme göre prediyabet durumundayım.  Benim kan şekerim aşırı derecede yüksek.  Bilgi: Sende diyabet hastalığı mevcut

In [8]:
len(df)

1000

In [9]:
eval_dataset = Dataset.from_pandas(df)

In [10]:
eval_dataset

Dataset({
    features: ['text'],
    num_rows: 1000
})

In [11]:
tokenizer = AutoTokenizer.from_pretrained(model_yolu)
model = AutoModelForCausalLM.from_pretrained(model_yolu)

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [12]:
# Eğitim argümanlarını tanımla (sadece değerlendirme için gerekli olanlar)
training_args = TrainingArguments(
    output_dir=model_yolu,  # Modelin kaydedileceği dizin
    report_to="none",  # Raporlama aracı
    per_device_eval_batch_size=1,  # Değerlendirme batch boyutu (azaldıkça vram kullanımı azalır)
    dataloader_drop_last=True,  # Son batch'ı atla
    eval_accumulation_steps=10,  # Değerlendirme adım sayısı (azaldıkça vram kullanımı azalır)

)

In [13]:
!pip install evaluate
!pip install rouge_score

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 3.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for rouge_score: filename=rouge_score-0.1.2-py3-none-any.whl size=24933 sha256=fd213c4a34abc913dbd3aef40b18f2283e901fbba0d03fcc73dd1710677da741
  Stored in directory: /root/.cache/pip/wheels/5f/dd/89/461065a73be61a532ff8599a28e9beef17985c9e9c31e541b4
Successfully built rouge_score


In [14]:
import evaluate

In [15]:
# Hazır metrikleri kullanarak compute_metrics fonksiyonunu tanımla
bleu = evaluate.load("bleu")
rouge = evaluate.load("rouge")
exact_match = evaluate.load("accuracy")  # EM için doğruluk metriğini kullanıyoruz
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)

    # Tahmin ve etiketleri metne dönüştür
    decoded_preds = tokenizer.batch_decode(predictions, skip_special_tokens=True)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

    # BLEU skoru hesaplama
    bleu_result = bleu.compute(predictions=decoded_preds, references=decoded_labels)

    # ROUGE skoru hesaplama
    rouge_result = rouge.compute(predictions=decoded_preds, references=decoded_labels)

    # Exact Match (EM) skoru hesaplama
    em_result = exact_match.compute(predictions=decoded_preds, references=decoded_labels)

    # Tüm metrikleri birleştirme
    metrics = {"bleu_sonuclari":bleu_result, "rouge": rouge_result["rouge1"], "exact_match": em_result["accuracy"]}
    return metrics

In [16]:
# Eğitici (Trainer) tanımla (yalnızca değerlendirme için gerekli olanlar)
trainer = SFTTrainer(
    model=model,
    tokenizer=tokenizer,
    args=training_args,
    eval_dataset=eval_dataset,
    dataset_text_field="text",
    compute_metrics=compute_metrics  # compute_metrics fonksiyonunu geçirin
)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_deprecation.py:100: FutureWarning: Deprecated argument(s) used in '__init__': dataset_text_field. Will not be supported from version '1.0.0'.

Deprecated positional argument(s) used in SFTTrainer, please use the SFTConfig to set these arguments instead.
  warnings.warn(message, FutureWarning)
/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1965: FutureWarning: `--push_to_hub_token` is deprecated and will be removed in version 5 of 🤗 Transformers. Use `--hub_token` instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/trl/trainer/sft_trainer.py:278: UserWarning: You didn't pass a `max_seq_length` argument to the SFTTrainer, this will default to 1024
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/trl/trainer/sft_trainer.py:307: UserWarning: You passed a `dataset_text_field` argument to the SFTTrainer, the value you passed will override the one in the `SFTConfig`.
  warnings.warn(


Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


In [ ]:
# Değerlendirmeyi başlat
results = trainer.evaluate()

In [ ]:
print(results)

In [ ]:
# Değerlendirme sonuçlarını kaydetme
results_file = os.path.join(model_yolu, "evaluation_results.json")
with open(results_file, "w") as f:
    json.dump(results, f)